***RAG with multiple sources and***
***Agent Tool based custom chatbot***

In [143]:
# Access the environment variables
import os
from langchain_openai import ChatOpenAI
#from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini" ,temperature=0.8)
# Access the environment variables
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


In [156]:
# Document loader (full directory - only docx)
from langchain_community.document_loaders import DirectoryLoader
path = '../GPT-4omini/data/'
docs_loader = DirectoryLoader(path, glob="**/*.docx",show_progress=True)
docs = docs_loader.load()


100%|██████████| 11/11 [00:01<00:00,  9.62it/s]


In [157]:
# Document loader (only pdf)
from langchain_community.document_loaders import PyPDFDirectoryLoader
pdf_loader = PyPDFDirectoryLoader(path, glob="**/*.pdf")
pdf_docs = pdf_loader.load()


Overwriting cache for 0 1974


In [158]:
# Combining both pdf & docs
total_docs = docs + pdf_docs
# Splitting into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(total_docs)


In [159]:
## Store into FAISS Vector Database
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
## Vector embedding for text chunks
db1 = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever1 = db1.as_retriever()


In [154]:
# URL loader & Text loader
from langchain_community.document_loaders import TextLoader
url_path = '../GPT-4omini/URLs.txt'
text_loader = TextLoader(url_path)
urls = text_loader.load()

from langchain_community.document_loaders import UnstructuredURLLoader
#urls = ["https://www.aspiresys.com/articles/composable-commerce-build-buy-strategy/index.html",
        #"https://blog.aspiresys.com/retail/monoliths-to-composable-commerce/",]
#url_loader = UnstructuredURLLoader(urls=urls)
# Split the combined text into individual URLs
url_loader = UnstructuredURLLoader(urls=urls[0].page_content.splitlines())
data = url_loader.load()
# Splitting into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
online_data=text_splitter.split_documents(data)
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
## Vector embedding for text chunks
db2 = FAISS.from_documents(online_data, OpenAIEmbeddings())
retriever2 = db2.as_retriever()


In [155]:
urls[0].page_content.splitlines()


['https://www.ignitiv.com/from-monolith-to-composable-commerce/',
 'https://commercetools.com/blog/from-monolith-to-microservices-migrating-to-a-composable-commerce-architecture-in-only-7-steps',
 'https://alokai.com/blog/from-monolith-to-composable-commerce-the-ultimate-migration-guide',
 'https://fimple.co.uk/how-to-convert-monolithic-architecture-to-microservices-architecture-strangler-pattern/',
 'https://brainhub.eu/library/monolith-to-microservices-using-strangler-pattern',
 'https://dzone.com/articles/monolith-to-microservices-with-the-strangler-patte',
 'https://samirbehara.com/2018/09/10/monolith-to-microservices-using-strangler-pattern/',
 'https://amplication.com/blog/monoliths-to-microservices-using-the-strangler-pattern',
 'https://insights.sei.cmu.edu/blog/8-steps-for-migrating-existing-applications-to-microservices/',
 'https://www.bolt.com/thinkshop/ecommerce-replatforming-migration',
 'https://www.aspiresys.com/composable-commerce-navigating-build/',
 'https://blog.asp

In [160]:
# Create Retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool1 = create_retriever_tool(
    retriever1,
    "doc_search",
    "Search for information from documents, you must use this tool!",
)
retriever_tool2 = create_retriever_tool(
    retriever2,
    "url_search",
    "Search for information from the internet, you must use this tool!",
)
#tools
tools = [retriever_tool1, retriever_tool2]


In [183]:
# create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
# Adapted from https://smith.langchain.com/hub/jacob/tool-calling-agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You need to use tools for every user's query.Do not answer anything beyond the scope of the two tools.",
            #First search into retrieve_1 and if information is not found then search into retrieve_2",
        ),
        ("placeholder", "{messages}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools = [retriever_tool1, retriever_tool2]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [184]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What are the best ways to avoid the negative consequences of vendor lock-in.?")]})
print(response)




> Entering new AgentExecutor chain...

Invoking: `url_search` with `{'query': 'how to avoid vendor lock-in consequences'}`


Avoid contracts containing clauses that make it difficult or impossible to switch providers. Before signing a contract, be sure to read it carefully and understand all the terms and conditions. If there are any clauses that you do not understand, ask the vendor to explain them to you.

Be prepared to switch providers. If you are not happy with the product or service that you are using, be prepared to switch providers. This may involve incurring some costs, but it may be worth it in the long run if it means that you can get a better product or service.

Vendor lock-in is a serious problem that can have a number of negative consequences for customers. By taking steps to avoid vendor lock-in, customers can protect themselves from these risks and ensure that they are getting the best possible value for their money.

Reduced flexibility. Vendor lock-in can reduce th